# Hypothesis Testing

In [1]:
import numpy as np
import pandas as pd

import pingouin

from scipy.stats import norm, poisson, invgamma, f
from scipy.stats import ttest_1samp, ttest_ind, bartlett, levene, f_oneway

In [2]:
# Significance level
ALPHA = 0.05

---

In [3]:
class anova():
    def __init__(self, data_group, variable="Y", alpha=ALPHA):
        # data_group must contain three columns:
        # count, the number of observations in the group
        # mean, observed average in the group
        # var, observed variance in the group

        J = len(data_group)
        n_tot = sum(data_group[variable]["count"])
        n_bar = n_tot / J

        s2_bar = data_group[variable]["var"] / data_group[variable]["count"]
        phi2 = 1/sum(1 / s2_bar)

        self.mu = phi2 * sum(data_group[variable]["mean"] / s2_bar)

        # Sum of squares between and within groups
        SSB = sum(data_group[variable]["count"] * (data_group[variable]["mean"] - self.mu)**2)
        SSW = sum((data_group[variable]["count"]-1) * data_group[variable]["var"])

        # Degrees of freedom between and within groups
        dfB = J - 1
        dfW = n_tot - J

        # Mean squares between and within groups
        MSB = SSB / dfB
        MSW = SSW / dfW

        self.sigma = np.sqrt(MSW)

        # F-test
        F_anova = MSB / MSW

        # P-value
        P_value = f.sf(F_anova, dfB, dfW)

        # ANOVA table
        self.anova_table = pd.DataFrame(
            {"SS":[SSB, SSW], "df":[dfB, dfW], "MS":[MSB, MSW], "F":[F_anova, np.nan], "PValue":[P_value, np.nan]},
            index=["Group", "Within"]
        )

        # Estimate parameters
        if P_value < alpha:
            self.theta = data_group[variable]["mean"]
            self.tau = np.sqrt((MSB - MSW) / n_bar)
        else:
            self.theta = self.mu
            self.tau = 0

---

Simulate 6 groups, each one with its own amount of observations, and following a hierarchical structure.

$$
\begin{align*}
Y_{ij} &\sim\textsf{Normal}(\theta_j,\sigma_j^2) \\
\theta_j &\sim\textsf{Normal}(\mu, \tau^2) \\
\sigma_j^2 &\sim\textsf{Inverse-}\chi^2(\nu, \rho^2)
\end{align*}
$$

In [4]:
# Number of groups
J = 6

# Size of each group
np.random.seed(111)
N = poisson.rvs(3, size=J, loc=10) 

# Parameters of the hierarchical model
MU  = -9
TAU = 1
RHO = 3
NU  = 4

# Mean and variance of each group
np.random.seed(111)
THETA  = norm.rvs(size=J, loc=MU, scale=TAU)
SIGMA2 = invgamma.rvs(size=J, a=NU/2, scale=NU/2*RHO**2)
SIGMA = np.sqrt(SIGMA2)

# Observations for each group
Y = norm.rvs(size=N[0], loc=THETA[0], scale=SIGMA[0])
Dat = pd.DataFrame(np.array([Y, [0]*N[0]]).T)

for j in range(1,J):
    Y = norm.rvs(size=N[j], loc=THETA[j], scale=SIGMA[j])
    DatAux = pd.DataFrame(np.array([Y, [j]*N[j]]).T)
    Dat = pd.concat([Dat, DatAux])

Dat = Dat.rename(columns={0:"Y", 1:"Group"})
Dat["Group"] = Dat["Group"].astype(int)

In [5]:
DatGroup = Dat.groupby("Group").agg(["count", "mean", "var"]).reset_index()

In [6]:
DatGroup.round(2)

Group     Y              
        count   mean    var
0     0    12 -10.15   1.86
1     1    12  -8.71  14.57
2     2    10  -8.42   3.44
3     3    12  -7.73  11.54
4     4    12 -12.22  79.30
5     5    14  -9.04   3.62

---

## One sample Student's t-test

### Comparing the mean of Group 0 against $\mu_0=-9$

In [7]:
mu0 = -9

In [8]:
pingouin.ttest(Dat[Dat["Group"]==0]["Y"], mu0)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.914851,11,two-sided,0.014066,"[-11.02, -9.28]",0.841445,4.492,0.756126


In [9]:
ttest_1samp(Dat[Dat["Group"]==0]["Y"], mu0)

TtestResult(statistic=-2.914850712740165, pvalue=0.014066446537359667, df=11)

The $p$-value is so low, that we can reject the hypothesis that $\mu_0=-9$

### Comparing the mean of Group 5 against $\mu_0=-9$

In [10]:
pingouin.ttest(Dat[Dat["Group"]==5]["Y"], mu0)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.081844,13,two-sided,0.936017,"[-10.14, -7.94]",0.021874,0.271,0.050662


In [11]:
ttest_1samp(Dat[Dat["Group"]==5]["Y"], mu0)

TtestResult(statistic=-0.08184406937964947, pvalue=0.9360174088882173, df=13)

The $p$-value is so high, that we cannot reject the hypothesis that $\mu_5=-9$

---

## Testing equal variance between two groups

### Comparing Group 2 and Group 5

In [12]:
FTest = (DatGroup[DatGroup["Group"]==2]["Y"]["var"]).values / (DatGroup[DatGroup["Group"]==5]["Y"]["var"]).values
n1 = DatGroup[DatGroup["Group"]==2]["Y"]["count"]
n2 = DatGroup[DatGroup["Group"]==5]["Y"]["count"]

f.sf(FTest, n1, n2)

array([0.52220779])

### Bartlett and Levene tests

Bartlett's and Levene's tests are popular to test equal variance between groups

In [13]:
bartlett(Dat[Dat["Group"]==5]["Y"], Dat[Dat["Group"]==2]["Y"])

BartlettResult(statistic=0.007122692067720255, pvalue=0.9327415533221795)

In [14]:
levene(Dat[Dat["Group"]==5]["Y"], Dat[Dat["Group"]==2]["Y"])

LeveneResult(statistic=0.1411107486371548, pvalue=0.7107769681238512)

We cannot reject the hypothesis of equal variances.

### Comparing Group 0 and Group 4

We will test the hypothesis that the variance of the groups 0 and 4 are equal

In [15]:
FTest = (DatGroup[DatGroup["Group"]==4]["Y"]["var"]).values / (DatGroup[DatGroup["Group"]==0]["Y"]["var"]).values
n1 = DatGroup[DatGroup["Group"]==4]["Y"]["count"]
n2 = DatGroup[DatGroup["Group"]==0]["Y"]["count"]

f.sf(FTest, n1, n2)

array([6.10386152e-08])

In [16]:
bartlett(Dat[Dat["Group"]==0]["Y"], Dat[Dat["Group"]==4]["Y"])

BartlettResult(statistic=25.374224916868826, pvalue=4.7218519878538826e-07)

In [17]:
levene(Dat[Dat["Group"]==0]["Y"], Dat[Dat["Group"]==4]["Y"])

LeveneResult(statistic=6.152892806692913, pvalue=0.021251787316881675)

We reject the hypothesis of equal variances.

In conclusion, we can act as if Groups 2 and 5 have the same variance. But Groups 0 and 4 have different variances.

---

## Testing equal mean between two groups

### Comparing Group 0 and Group 5

In [18]:
DatGroup

Group     Y                      
        count       mean        var
0     0    12 -10.148234   1.862126
1     1    12  -8.706604  14.571669
2     2    10  -8.423547   3.435911
3     3    12  -7.731961  11.539293
4     4    12 -12.217278  79.296169
5     5    14  -9.041636   3.623141

In [19]:
pingouin.ttest(Dat[Dat["Group"]==0]["Y"], Dat[Dat["Group"]==5]["Y"])

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.719903,23.344629,two-sided,0.098682,"[-2.44, 0.22]",0.659436,1.05,0.363095


In [20]:
ttest_ind(Dat[Dat["Group"]==0]["Y"], Dat[Dat["Group"]==5]["Y"])

TtestResult(statistic=-1.6762576764168238, pvalue=0.10666807353971323, df=24.0)

We have slight evidence against the hypothesis of equal means

### Comparing Group 0 and Group 4

In [21]:
DatGroup

Group     Y                      
        count       mean        var
0     0    12 -10.148234   1.862126
1     1    12  -8.706604  14.571669
2     2    10  -8.423547   3.435911
3     3    12  -7.731961  11.539293
4     4    12 -12.217278  79.296169
5     5    14  -9.041636   3.623141

In [22]:
pingouin.ttest(Dat[Dat["Group"]==0]["Y"], Dat[Dat["Group"]==4]["Y"])

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.795599,22,two-sided,0.434766,"[-3.32, 7.46]",0.324802,0.47,0.118586


In [23]:
ttest_ind(Dat[Dat["Group"]==0]["Y"], Dat[Dat["Group"]==4]["Y"])

TtestResult(statistic=0.7955985692871433, pvalue=0.4347661300165786, df=22.0)

We cannot reject the hypothesis of equal means.

### Comparing Group 0 and Group 5

But, before we concluded that these groups have different variance. We can take this into account

In [ ]:
pingouin.ttest(Dat[Dat["Group"]==0]["Y"], Dat[Dat["Group"]==4]["Y"], correction=True) #correction=True to consider different variances

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.795599,11.516345,two-sided,0.442356,"[-3.62, 7.76]",0.324802,0.47,0.118586


In [ ]:
ttest_ind(Dat[Dat["Group"]==0]["Y"], Dat[Dat["Group"]==4]["Y"], equal_var=False) #equal_var=False to consider different variances

TtestResult(statistic=0.7955985692871433, pvalue=0.4423556292510572, df=11.51634521273693)

---

## One-way ANOVA

### By hand

In [26]:
AnovaTable = anova(DatGroup)

In [27]:
AnovaTable.anova_table

,SS,df,MS,F,PValue
Group,152.908835,5,30.581767,1.604467,0.171163
Within,1257.985860,66,19.060392,NaN,NaN


In this case, we do not reject the hypothesis of equal mean in the groups.

In [28]:
# Between groups mean and deviation
AnovaTable.mu, AnovaTable.tau

(-9.31570192315129, 0)

In [29]:
# Mean for each group
AnovaTable.theta

-9.31570192315129

In [30]:
# Common within group deviation
AnovaTable.sigma

4.365820865052846

### Using pingouin

In [31]:
pingouin.anova(dv="Y", between="Group", data=Dat, detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,Group,152.451747,5,30.490349,1.599671,0.172473,0.108088
1,Within,1257.985860,66,19.060392,NaN,NaN,NaN


### Using scipy

Testing equal mean for all the groups

In [32]:
f_oneway(
    Dat[Dat["Group"]==0]["Y"],
    Dat[Dat["Group"]==1]["Y"],
    Dat[Dat["Group"]==2]["Y"],
    Dat[Dat["Group"]==3]["Y"],
    Dat[Dat["Group"]==4]["Y"],
    Dat[Dat["Group"]==5]["Y"]
)

F_onewayResult(statistic=1.599670646695831, pvalue=0.17247260234129147)